In [2]:
%%writefile Functions/data_setup.py


"""
Contains the functionality for creating PyTorch DataLoaders for image classification data.
"""

import os

import torch
import torchvision
from torchvision import datasets, transforms 
from torch.utils.data import DataLoader 

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    train_transform: transforms.Compose,
    test_transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
    ):
    
    """ Creates Training and Testing DataLoaders.
    
    Takes in a training directory and testing directory path and 
    turns them into PyTorch Datasets and then into PyTorch DataLoaders.
    
    Args:
        train_dir: Path to training directory.
        test_dir: Path to testing directory.
        transform: torchvision transforms to perform on training and testing data.
        batch_size: number of samples per batch in each of the DataLoaders.
        num_workers: An integer for number of workers per DataLoader 
    
    Returns:
        A tuple of (train_dataloader, test_dataloader , class_names).
        where class_names is a list of the target classes.
    
    Example Usage:
        train_dataloader, test_dataloader, class_names = create_dataLoader(
                                                                           train_dir = path/to/train_dir,
                                                                           test_dir = path/to/test_dir,
                                                                           train_transform = some_transform,
                                                                           test_tansform = some_transform,
                                                                           batch_size = 32,
                                                                           num_workers = 2
                                                                           )
    """
    
    # Use ImageFolder to create Dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=train_transform)
    test_data = datasets.ImageFolder(test_dir, transform=test_transform)
    
    # Get class names :
    class_names = train_data.classes
    
    # Turn image into DataLoaders 
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
        )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,               # Don't need to shuffle test data
        num_workers=num_workers,
        pin_memory=True
        )
        
    
    return train_dataloader, test_dataloader, class_names 


    
    
    
        

Writing Functions/data_setup.py


In [3]:
%%writefile Functions/model_builder.py
"""
Contains PyTorch model to instantiate a TinyVGG model
"""

import torch
from torch import nn

class TinyVGG(nn.Module):
    """
    Creates the TinyVGG architecture.
    
    Replicates the TinyVGG architecture from the CNN Explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/
    
    Args:
        input_shape: An integer indicating number of input channels.
        hidden_units: An integer indicating number of hidden units(neurons) between layers.
        output_shape: An integer indicating number of output channels.
    """
    
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                        out_channels=hidden_units,
                        kernel_size=3,
                        stride=1,
                        padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                        out_channels=hidden_units,
                        kernel_size=3,
                        stride=1,
                        padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                           stride=2)              # the stride is always equal to kernel size in MaxPool2d layer 
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                        out_channels=hidden_units,
                        kernel_size=3,
                        stride=1,
                        padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                        out_channels=hidden_units,
                        kernel_size=3,
                        stride=1,
                        padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                           stride=2)              # the stride is always equal to kernel size in MaxPool2d layer 
        ) 
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*13*13,
                      out_features=output_shape)
        )
        
    def forward(self, x: torch.Tensor):
            return self.classifier(self.conv_block_2(self.conv_block_1(x)))              # leaverages the benefits of operation fusion
       
        

Writing Functions/model_builder.py


In [4]:
%%writefile Functions/train.py
"""
Trains a PyTorch image classification model using device agnostic code 
"""

import os 
import torch
import torchvision
from torchvision import transforms

import data_setup, engine, model_builder, utils, Get_data


# Setup the HYPERPARAMETERS 
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup the directories 
train_dir = Get_data.IMAGE_PATH / "train"
test_dir = Get_data.IMAGE_PATH / "test"

# Setup target device(device agnostic code)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms

train_transform_model = transforms.Compose([
                                      transforms.Resize(size=(64,64)),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.TrivialAugmentWide(num_magnitude_bins=32),
                                      transforms.ToTensor()
                                   ])
test_transform_model = transforms.Compose([
                                    transforms.Resize(size=(64,64)),    
                                    transforms.ToTensor()
                                   ])

# Create DataLoaders with the help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               train_transform=train_transform_model,
                                                                               test_transform=test_model,
                                                                               batch_size=BATCH_SIZE,
                                                                               num_workers=os.cpu_count()
                                                                              )

# Create a model with the help of model_builder.py
Model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=HIDDEN_UNITS,
                              output_shape=len(class_names)
                             ).to(device)

# Set Loss and optimizer function
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with the help of engine.py
engine.train(model=Model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_func=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device
            )

# Save the model with the help from utils.py
utils.save_model(model=Model,
                 target_dir="D:\PROJECT",
                 model_name="image_classfication_TinyVGG_model.pth")



Writing Functions/train.py


In [5]:
%%writefile Functions/utils.py
"""
Creates various utility functions for PyTorch mdoel training and testing
"""

import torch
import torch.nn as nn
from pathlib import Path

def accuracy(output, target):
    # Get the index of max log probability
    pred = output.max(1, keep_dim=True)[1]
    return pred.eq(target.view_as(pred)).cpu().float().mean()

def save_model(model: nn.Module,
               target_dir: str, 
               model_name: str):
    """
        Saves a PyTorch model to target directory.
        
        Args:
        model: A PyTorch model to save.
        target_dir: A destination directory.
        model_name: A filename for the saved model. Should include either ".pth" or ".pt" as the file extension.
        
        Example usage:
            save_model(model=model_0,
                       target_dir="model5"
                       model_name="going_modular.pth"
                       
    """
    
    # Create target directory:
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                          exist_ok=True)
    
    # Create model save path 
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'."
    model_save_path = target_dir_path / model_name
    
    # Save the model state_dict()
    print(f"[INFO] saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
               f=model_save_path)
    
    

Writing Functions/utils.py


In [6]:
%%writefile Functions/engine.py
"""
Creates functions for training and testing a PyTorch model.
"""

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_func: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    
    """
    Trains a PyTorch model for a single epoch.
    
    Turns a target PyTorch model to training mode and then runs 
    through all of the required training steps(forward pass, loss calculation, optimization)
    
    Args:
        model: A PyTorch model to be trained.
        dataloader: A DataLoader instance for the model to be trained on.
        loss_func: A PyTorch loss function to minimize.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        device: A target device to compute on.
        
    Returns: 
        A tuple of training loss and training accuracy metrics.
        In the form(train_loss, train_accuracy)
        
    """
    # Put model in train mode 
    model.train()
    
    # Setup loss and accuracy values 
    train_loss, train_accuracy = 0, 0
    
    # Loop through data loader batches
    for batch, (data, label) in enumerate(dataloader):
        # send data to target device
        data, label = data.to(device), label.to(device)
        
        #1. Forward pass
        y_logit = model(data)
        
        #2. loss calculation and accumulation
        loss = loss_func(y_logit, label)
        train_loss += loss.item()
        
        #3. Optimizer zero grad
        optimizer.zero_grad()
        
        #4. Loss backwards (Backpropagation)
        loss.backward()
        
        #5. Optimizer step (Gradient descent)
        optimizer.step()
        
        #6. calculate and accumulate accuracy metric across all batches 
        y_pred_class = torch.argmax(torch.softmax(y_logit, dim=1), dim=1)
        train_accuracy += (y_pred_class == label).sum().item()/len(y_logit)
    
    # Adjust metrics to get average loss and accuracy
    train_loss = train_loss / len(dataloader)
    train_accuracy =  train_accuracy / len(dataloader)
    
    return train_loss, train_accuracy
        
    
    

    
def test_step(model: nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_func: torch.nn.Module,
               device: torch.device) -> Tuple[float, float]:
    
    """
    Tests a PyTorch model for a single epoch.
    
    Turns a target PyTorch model to 'eval' mode and then runs 
    through the forward pass, loss calculation on a testing dataset.
        
    Args:
        model: A PyTorch model to be tested.
        dataloader: A DataLoader instance for the model to be tested on.
        loss_func: A PyTorch loss function to minimize.
        device: A target device to compute on.
        
    Returns: 
        A tuple of testing loss and testing accuracy metrics.
        In the form(test_loss, test_accuracy)
        
    """
    # Put model in train mode 
    model.eval()
    
    # Setup loss and accuracy values 
    test_loss, test_accuracy = 0, 0
    
    # Turn on the inference context manager 
    with torch.inference_mode():
        
        # Loop through data loader batches
        for batch, (data, label) in enumerate(dataloader):
            # send data to target device
            data, label = data.to(device), label.to(device)

            #1. Forward pass
            y_logit = model(data)

            #2. loss calculation and accumulation
            loss = loss_func(y_logit, label)
            test_loss += loss.item()

            #3. calculate and accumulate accuracy metric across all batches 
            y_pred_class = torch.argmax(torch.softmax(y_logit, dim=1), dim=1)
            test_accuracy += (y_pred_class == label).sum().item()/len(y_logit)

        # Adjust metrics to get average loss and accuracy
        test_loss = test_loss / len(dataloader)
        test_accuracy =  test_accuracy / len(dataloader)

        return test_loss, test_accuracy
    
    

def train(model: nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_func: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    
    """
        Trains and tests a PyTorch model.
        
        Passes a target PyTorch models through train_step() and test_step()
        functions for a number of epochs, training and testing the model in the same epoch loop
        
        Calculates, prints and stores evaluation metrics throughout.
        
        Args:
        
        model: A PyTorch model to be trained.
        train_dataloader: A DataLoader instance for the model to be trained on.
        test_dataloader: A DataLoader instance for the model to be tested on.
        loss_func: A PyTorch loss function to minimize loss on both datasets.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        epochs: An integer indicating how many epochs to train on.
        device: A target device to compute on.
        
        Returns:
        
        A dictionary of training and testing loss as well as training 
        and testing accuracy metrics. Each metric has a value in a list for each epoch.
        In the form: {train_loss: [...],
                      train_accuracy: [...],
                      test_loss: [...],
                      test_loss: [...]
                      }
    """
    # Create an empty dictionary 
    results = {"train_loss": [],
               "train_accuracy": [],
               "test_loss": [],
               "test_accuracy": []
              }
    
    # Loop through training and testing steps for number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_accuracy = train_step(model=model,
                                                dataloader=train_dataloader,
                                                loss_func=loss_func,
                                                optimizer=optimizer,
                                                device=device)
        test_loss, test_accuracy = test_step(model=model,
                                             dataloader=test_dataloader,
                                             loss_func=loss_func,
                                             device=device)
        # print out what's happening 
        print(f"Epoch: {epoch+1} |"
              f"Training loss: {train_loss:.4f} |"
              f"Training accuracy: {train_accuracy:.4f} |"
              f"Testing loss: {test_loss:.4f} |"
              f"Testing accuracy: {test_accuracy:.4f} |"
             )
        
        # update the results dictionary
        results["train_loss"].append(train_loss)
        results["train_accuracy"].append(train_accuracy)
        results["test_loss"].append(test_loss)
        results["test_accuracy"].append(test_accuracy)
    
    return results
        

Writing Functions/engine.py
